In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import (
    f1_score, roc_auc_score, precision_score, recall_score,
    confusion_matrix, classification_report, matthews_corrcoef, roc_curve, auc
)
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from skopt import BayesSearchCV
from skopt.space import Real, Integer
import torch
import torch.nn as nn
import torch.optim as optim

class DropoutMLPClassifier(nn.Module):
    def __init__(self, input_size, hidden_layer_sizes, dropout_prob=0.2):
        super(DropoutMLPClassifier, self).__init__()
        layers = []
        prev_size = input_size
        for size in hidden_layer_sizes:
            layers.append(nn.Linear(prev_size, size))
            layers.append(nn.ReLU())
            layers.append(nn.Dropout(p=dropout_prob))
            prev_size = size
        layers.append(nn.Linear(prev_size, 1))
        self.network = nn.Sequential(*layers)

    def forward(self, x):
        return torch.sigmoid(self.network(x))

def load_data_from_excel(file_path):
    df = pd.read_excel(file_path)
    X = df.iloc[:, :-1].values
    y = df.iloc[:, -1].values  # (0 for intact cells, 1 for apoptotic cells)
    return X, y

file_path = 'path_to_your_file.xlsx'  # This part of the code enables the use of MS Excel file
X, y = load_data_from_excel(file_path)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

pipe = Pipeline([
    ('scaler', StandardScaler()),
    ('mlp', MLPClassifier(max_iter=500, random_state=42, early_stopping=True))
])

param_space = {
    'mlp__hidden_layer_sizes': [(50, 50), (100, 50), (100, 100)],
    'mlp__activation': ['tanh', 'relu'],
    'mlp__solver': ['adam', 'lbfgs'],
    'mlp__alpha': Real(1e-5, 1e-2, prior='log-uniform'),
    'mlp__learning_rate_init': Real(1e-4, 1e-1, prior='log-uniform'),
    'mlp__beta_1': Real(0.8, 0.999),
    'mlp__beta_2': Real(0.9, 0.999),
}

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
bayes_search = BayesSearchCV(pipe, param_space, cv=cv, n_iter=32, scoring='f1', n_jobs=-1, verbose=2, random_state=42)
bayes_search.fit(X_train, y_train)


y_pred = bayes_search.predict(X_test)
y_prob = bayes_search.predict_proba(X_test)[:, 1]


f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_prob)
mcc = matthews_corrcoef(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

print("Classification Report:")
print(classification_report(y_test, y_pred))

print(f"F1 Score: {f1}")
print(f"ROC AUC: {roc_auc}")
print(f"MCC: {mcc}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")

conf_matrix = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(conf_matrix)

fpr, tpr, thresholds = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC)')
plt.legend(loc="lower right")
plt.show()

print("The Best Hyperparameters are:")
print(bayes_search.best_params_)
